<a href="https://colab.research.google.com/github/plz77/LLM-Fine-Tuning/blob/main/LLM_Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Project 1: Fine-Tuning LLM

--------------------------------------------------------------------------------
Modul Pelatihan: Fine-Tuning LLM (GPT-2) untuk Edukasi
--------------------------------------------------------------------------------
Halo semuanya! Hari ini kita akan belajar cara "mengajari" sebuah kecerdasan
buatan (LLM) agar lebih pintar dalam konteks pendidikan di Indonesia.

Apa itu Fine-Tuning?
Ibarat kita punya asisten yang pintar bahasa Inggris secara umum (GPT-2),
kita akan memberinya "kursus kilat" agar ia memahami istilah kurikulum
dan metode belajar di Indonesia.

Istilah Penting:
1. Epoch: Berapa kali model membaca seluruh dataset (buku) dari awal sampai akhir.
2. Tokenisasi: Proses mengubah kata menjadi angka agar komputer bisa mengerti.
3. Inference: Tahap pengujian atau penggunaan model setelah ia selesai belajar.
--------------------------------------------------------------------------------


In [ ]:
import os
import torch
from transformers import (
    GPT2Tokenizer,
    GPT2LMHeadModel,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling,
    pipeline
)
from datasets import Dataset

# Step 1: Persiapan Lingkungan (Environment)
os.environ["CUDA_VISIBLE_DEVICES"] = ""

def train_edu_model():
    print("--- Memulai Modul Pembelajaran Fine-Tuning ---")

    # Step 2: Memilih Model Dasar (Base Model)
    model_name = "gpt2"
    print(f"1. Memuat model dasar: {model_name}...")

    try:
        tokenizer = GPT2Tokenizer.from_pretrained(model_name)
        tokenizer.pad_token = tokenizer.eos_token
        model = GPT2LMHeadModel.from_pretrained(model_name).to("cpu")
    except Exception as e:
        print(f"Gagal memuat model: {e}")
        return # Sekarang return berada di dalam fungsi yang benar

    # Step 3: Menyiapkan Dataset Pendidikan (Dataset diperkaya)
    print("2. Menyiapkan materi dataset pendidikan...")
    data_samples = [
        {"text": "Pendidikan di Indonesia menggunakan Kurikulum Merdeka untuk fleksibilitas belajar."},
        {"text": "Kecerdasan Buatan atau AI adalah subjek penting untuk masa depan siswa Gen Z."},
        {"text": "Algoritma adalah urutan langkah logis untuk menyelesaikan masalah komputasi."},
        {"text": "Machine Learning memungkinkan komputer belajar dari data tanpa pemrograman."},
        {"text": "Python adalah bahasa pemrograman populer untuk pengembangan kecerdasan buatan."},
        {"text": "Literasi digital sangat penting bagi guru dan siswa di era modern."},
        {"text": "Cloud Computing mempermudah akses penyimpanan data pendidikan secara online."},
        {"text": "Siswa belajar coding untuk meningkatkan kemampuan logika dan problem solving."},
        {"text": "Teknologi masa depan akan sangat bergantung pada inovasi di bidang AI."},
        {"text": "Guru berperan sebagai fasilitator dalam ekosistem pendidikan digital."},
        {"text": "Pendidikan masa depan mengutamakan kolaborasi antara manusia dan teknologi AI."},
        {"text": "Kurikulum Merdeka memberikan kebebasan bagi guru untuk berinovasi di kelas."},
        {"text": "Siswa harus menguasai keterampilan digital untuk bersaing di pasar kerja global."},
        {"text": "Teknologi informatika menjadi pilar utama dalam transformasi sekolah digital."},
        {"text": "Metode pembelajaran berbasis proyek membantu siswa berpikir lebih kritis."}
    ]

    raw_dataset = Dataset.from_list(data_samples)

    # Step 4: Tokenisasi
    def tokenize_function(examples):
        return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=64)

    print("3. Proses tokenisasi data...")
    tokenized_datasets = raw_dataset.map(tokenize_function, batched=True, remove_columns=["text"])
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

    # Step 5: Konfigurasi Pelatihan
    training_args = TrainingArguments(
        output_dir="./gpt2-edu-result",
        overwrite_output_dir=True,
        num_train_epochs=10,
        per_device_train_batch_size=1,
        no_cuda=True,
        report_to="none"
    )

    # Step 6: Proses Belajar (Training)
    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=tokenized_datasets,
    )

    print("\n--- Proses Fine-Tuning Dimulai ---")
    try:
        trainer.train()
        print("Training selesai!")

        # Menyimpan model hasil pelatihan
        model.save_pretrained("./model-finetuning-kita")
        tokenizer.save_pretrained("./model-finetuning-kita")
        print("Model berhasil disimpan di folder: ./model-finetuning-kita")

        # Step 7: Ujian Akhir (Inference)
        print("\n--- Tahap Pengujian (Inference) ---")
        generator = pipeline(
            'text-generation',
            model="./model-finetuning-kita",
            tokenizer="./model-finetuning-kita",
            device=-1
        )

        prompt = "Pendidikan masa depan"
        print(f"Prompt: {prompt}")

        output = generator(
            prompt,
            max_new_tokens=25,
            do_sample=True,
            temperature=0.7,
            top_k=50
        )

        print("\nHasil Generasi AI:")
        print(f"{output[0]['generated_text']}")

    except Exception as e:
        print(f"\nTerjadi kesalahan selama proses: {e}")

if __name__ == "__main__":
    train_edu_model()

--- Memulai Modul Pembelajaran Fine-Tuning ---
1. Memuat model dasar: gpt2...
2. Menyiapkan materi dataset pendidikan...
3. Proses tokenisasi data...


Map:   0%|          | 0/15 [00:00<?, ? examples/s]


--- Proses Fine-Tuning Dimulai ---


Step,Training Loss


Training selesai!


Device set to use cpu


Model berhasil disimpan di folder: ./model-finetuning-kita

--- Tahap Pengujian (Inference) ---
Prompt: Pendidikan masa depan

Hasil Generasi AI:
Pendidikan masa depan siswa berperan pendidikan pengembangan untuk meningkatkan kecerdasan.
